<a href="https://colab.research.google.com/github/DonghaeSuh/NLP_Pytorch/blob/main/Model/Seq2Seq_Chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 간단한 전처리
- 특수 문자 제거
- 특수 토큰 추가 MAX_LENGTH인 25에 맞춰서 시작 토큰과 끝 토큰을 추가 -> 이후 field에서 추가해서 사용
- decoder의 입력에 따른 예측값에 해당하는 dataset을 마련

In [ ]:
import pandas as pd

In [ ]:
ls

drive/  sample_data/


In [ ]:
cd drive/MyDrive/Pytorch\ NLP/data_in/Chatbot

/content/drive/MyDrive/Pytorch NLP/data_in/Chatbot


In [ ]:
ls

ChatbotData.csv  Mecab-ko-for-Google-Colab/


In [ ]:
df=pd.read_csv('ChatbotData.csv')
df.head()

,Q,A,label
0,12시 땡!,하루가 또 가네요.,0
1,1지망 학교 떨어졌어,위로해 드립니다.,0
2,3박4일 놀러가고 싶다,여행은 언제나 좋죠.,0
3,3박4일 정도 놀러가고 싶다,여행은 언제나 좋죠.,0
4,PPL 심하네,눈살이 찌푸려지죠.,0


### 특수 문자 제거

In [ ]:
df['Q']=df['Q'].str.replace('[^ a-zA-Z0-9가-힣]',"")
df['A']=df['A'].str.replace('[^ a-zA-Z0-9가-힣]',"")
df.head()

<ipython-input-90-21d281052103>:1: FutureWarning: The default value of regex will change from True to False in a future version.
  df['Q']=df['Q'].str.replace('[^ a-zA-Z0-9가-힣]',"")
<ipython-input-90-21d281052103>:2: FutureWarning: The default value of regex will change from True to False in a future version.
  df['A']=df['A'].str.replace('[^ a-zA-Z0-9가-힣]',"")


,Q,A,label
0,12시 땡,하루가 또 가네요,0
1,1지망 학교 떨어졌어,위로해 드립니다,0
2,3박4일 놀러가고 싶다,여행은 언제나 좋죠,0
3,3박4일 정도 놀러가고 싶다,여행은 언제나 좋죠,0
4,PPL 심하네,눈살이 찌푸려지죠,0


In [ ]:
def decoder_input(sent,Max_len):
  sent=sent.split()
  if len(sent)>Max_len-2: # Max_length=25
    sent=sent[:Max_len-2]
    sent.append('<eos>')
    sent[:0]=['<sos>']
  else:
    sent.append('<eos>')
    sent[:0]=['<sos>']
  
  return " ".join(sent)

In [ ]:
def decoder_output(sent):
  sent=sent.split()
  sent.pop(0)
  return " ".join(sent)

In [ ]:
# 알고보니 torchtext의 필드정의에서 <sos>, <eos> token 추가를 지원했다. => 이럴 경우 굳이 미리 추가해놓고 준배해줄 필요가 없다.
df['label']=df['A']
temp=pd.Series(df['Q'].tolist()+df['A'].tolist()).astype(str)
temp.to_csv('vocab_word.csv',index=False)
df.head()

,Q,A,label
0,12시 땡,하루가 또 가네요,하루가 또 가네요
1,1지망 학교 떨어졌어,위로해 드립니다,위로해 드립니다
2,3박4일 놀러가고 싶다,여행은 언제나 좋죠,여행은 언제나 좋죠
3,3박4일 정도 놀러가고 싶다,여행은 언제나 좋죠,여행은 언제나 좋죠
4,PPL 심하네,눈살이 찌푸려지죠,눈살이 찌푸려지죠


In [ ]:
df.to_csv('2_Fine_ChatbotData.csv',index=False)

In [ ]:
decoder_input('안녕하세요 저는 사람입니다',4)

'<sos> 안녕하세요 저는 <eos>'

In [ ]:
decoder_output('<sos> 안녕하세요 저는 <eos>')

'안녕하세요 저는 <eos>'

In [ ]:
df['A']=df['A'].apply(lambda x : decoder_input(x,25))
df.head()

,Q,A,label
0,12시 땡,<sos> 하루가 또 가네요 <eos>,0
1,1지망 학교 떨어졌어,<sos> 위로해 드립니다 <eos>,0
2,3박4일 놀러가고 싶다,<sos> 여행은 언제나 좋죠 <eos>,0
3,3박4일 정도 놀러가고 싶다,<sos> 여행은 언제나 좋죠 <eos>,0
4,PPL 심하네,<sos> 눈살이 찌푸려지죠 <eos>,0


In [ ]:
df['label']=df['A'].apply(decoder_output)
df.head()

,Q,A,label
0,12시 땡,<sos> 하루가 또 가네요 <eos>,하루가 또 가네요 <eos>
1,1지망 학교 떨어졌어,<sos> 위로해 드립니다 <eos>,위로해 드립니다 <eos>
2,3박4일 놀러가고 싶다,<sos> 여행은 언제나 좋죠 <eos>,여행은 언제나 좋죠 <eos>
3,3박4일 정도 놀러가고 싶다,<sos> 여행은 언제나 좋죠 <eos>,여행은 언제나 좋죠 <eos>
4,PPL 심하네,<sos> 눈살이 찌푸려지죠 <eos>,눈살이 찌푸려지죠 <eos>


In [ ]:
df.to_csv('Fine_ChatbotData.csv',index=False)

# 데이터 불러오기 및 dataset,dataLoader 생성

In [ ]:
import pandas as pd

In [ ]:
ls

drive/  sample_data/


In [ ]:
cd drive/MyDrive/Pytorch\ NLP/data_in/Chatbot

/content/drive/MyDrive/Pytorch NLP/data_in/Chatbot


In [ ]:
ls

2_Fine_ChatbotData.csv  Fine_ChatbotData.csv        vocab_word.csv
ChatbotData.csv         Mecab-ko-for-Google-Colab/


In [ ]:
vocab_word=pd.read_csv('vocab_word.csv')
vocab_word.head()

,0
0,12시 땡
1,1지망 학교 떨어졌어
2,3박4일 놀러가고 싶다
3,3박4일 정도 놀러가고 싶다
4,PPL 심하네


In [ ]:
train_data=pd.read_csv('2_Fine_ChatbotData.csv')
train_data.head()

,Q,A,label
0,12시 땡,하루가 또 가네요,하루가 또 가네요
1,1지망 학교 떨어졌어,위로해 드립니다,위로해 드립니다
2,3박4일 놀러가고 싶다,여행은 언제나 좋죠,여행은 언제나 좋죠
3,3박4일 정도 놀러가고 싶다,여행은 언제나 좋죠,여행은 언제나 좋죠
4,PPL 심하네,눈살이 찌푸려지죠,눈살이 찌푸려지죠


tokenizer = 공백기준분리 str.split() 사용 예정


In [ ]:
## hyperparameter

BATCH_SIZE=2
MAX_LEN=25
EPOCH=50
UNIT=1024
EMB_DIM=256

### 필드 정의 및 Tabulardataset 생성

In [ ]:
pip install torchtext==0.6

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.2/64.2 kB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 26.7 MB/s eta 0:00:00
  Attempting uninstall: torchtext
    Found existing installation: torchtext 0.15.2
    Uninstalling torchtext-0.15.2:
      Successfully uninstalled torchtext-0.15.2


In [ ]:
from torchtext import data

In [ ]:
# vocab 전용
V=data.Field(sequential=True,
             use_vocab=True,
             tokenize=str.split,
             init_token='<sos>',
             eos_token='<eos>',
             fix_length=MAX_LEN)

In [ ]:
Q=data.Field(sequential=True,
             use_vocab=True,
             tokenize=str.split,
             batch_first=True,
             fix_length=MAX_LEN)

A=data.Field(sequential=True,
             use_vocab=True,
             init_token='<sos>',
             eos_token='<eos>',
             tokenize=str.split,
             batch_first=True,
             fix_length=MAX_LEN)
LABEL=data.Field(sequential=True,
                 use_vocab=True,
                 eos_token='<eos>',
                 tokenize=str.split,
                 batch_first=True,
                 fix_length=MAX_LEN)

In [ ]:
from torchtext.data import TabularDataset

In [ ]:
train_data=TabularDataset('2_Fine_ChatbotData.csv',format='csv',fields=[('question',Q),('answer',A),('label',LABEL)],skip_header=True)
vars(train_data[0])

{'question': ['12시', '땡'],
 'answer': ['하루가', '또', '가네요'],
 'label': ['하루가', '또', '가네요']}

In [ ]:
vocab_data=TabularDataset('vocab_word.csv',format='csv',fields=[('vocab_word',V)],skip_header=True)

### 사전 정의
- 3개의 Field는 하나의 사전을 공유
- vocab_word라는 Q와A의 모든 문장을 합친 데이터로부터 단어사전을 만든 다음에
- 해당 단어사전을 Q,A,LABEL에 전달할 예정

In [ ]:
V.build_vocab(vocab_data,min_freq=5)

Q.vocab=V.vocab
A.vocab=V.vocab
LABEL.vocab=V.vocab

In [ ]:
Q.vocab.stoi

In [ ]:
Q.vocab.itos[943]

'안녕'

In [ ]:
Q.vocab.stoi['안녕']

943

### 시드 고정 후 데이터 분리

In [ ]:
import random
import torch
SEED=123
random.seed(SEED)
torch.manual_seed(SEED)

train_data,val_data=train_data.split(split_ratio=0.9,random_state=random.seed(SEED))

### 데이터 로더 생성

In [ ]:
from torchtext.data import Iterator

In [118]:
train_iter=Iterator(train_data,batch_size=BATCH_SIZE)
val_iter=Iterator(val_data,batch_size=BATCH_SIZE)

In [ ]:
batch=next(iter(train_iter))
batch.question

tensor([[ 742,  202, 1854,    0,  211,    1,    1,    1,    1,    1,    1,    1,
            1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
            1],
        [  45,    0,    0, 2612,    1,    1,    1,    1,    1,    1,    1,    1,
            1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
            1]])

In [ ]:
batch.answer

tensor([[   2,    0,    0,    0,  858,  321,    0,   89,    3,    1,    1,    1,
            1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
            1],
        [   2,  978, 1768,  539,   89,    3,    1,    1,    1,    1,    1,    1,
            1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
            1]])

In [ ]:
batch.label

tensor([[   0,    0,    0,  858,  321,    0,   89,    3,    1,    1,    1,    1,
            1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
            1],
        [ 978, 1768,  539,   89,    3,    1,    1,    1,    1,    1,    1,    1,
            1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
            1]])

In [119]:
train_iter=Iterator(train_data,batch_size=BATCH_SIZE)
val_iter=Iterator(val_data,batch_size=BATCH_SIZE)

# 모델 구현

In [ ]:
## hyperparameter

BATCH_SIZE=2
MAX_LEN=25
EPOCH=50
UNIT=1024
EMB_DIM=256
device='cuda' if torch.cuda.is_available() else 'cpu'
vocab_size=len(V.vocab)

In [ ]:
## 모듈 불러오기
import torch
import torch.nn as nn

In [ ]:
class Encoder(nn.Module):
  def __init__(self,vocab_size,emb_dim,hidden_dim,dense_dim):
    super(Encoder,self).__init__()
    self.embedding=nn.Embedding(vocab_size,emb_dim)
    self.gru=nn.GRU(emb_dim,hidden_dim,batch_first=True)
    self.dense=nn.Linear(hidden_dim,dense_dim)

  def forward(self,x):
    x=self.embedding(x)  # (batch_size, Max_len, emb_dim)
    outputs,last_hidden_state=self.gru(x) # outputs: ( batch_size, Max_len, dense_dim ) last_hidden_state : ( 1, batch_size, hidden_dim )
    last_hidden_state=torch.squeeze(last_hidden_state,dim=0)
    return self.dense(outputs),self.dense(last_hidden_state) # outputs: ( batch_size, Max_len, dense_dim ) last_hidden_state : ( batch_size, hidden_dim )

In [ ]:
encoder=Encoder(vocab_size,EMB_DIM,UNIT,UNIT)
a,b=encoder(batch.question)
b=b.repeat(1,)

torch.Size([2, 1024])

In [ ]:
class Attention(nn.Module):
  def __init__(self,dense_dim):
    super(Attention,self).__init__()
    self.dense=nn.Linear(dense_dim,1)
    self.softmax=nn.Softmax(dim=1)

  def forward(self,outputs,last_hidden_state):
    last_hidden_state=last_hidden_state.view(2,1,1024)
    after_sum=outputs+last_hidden_state  # ( batch_size, Max_len, dense_dim ) + ( bacth_Size, 1, hidden_dim) => broadcasting
    after_dense=self.dense(after_sum) # ( batch_size, Max_len, 1 )
    after_softmax=self.softmax(after_dense) # ( batch_size, Max_len, 1 )
    context_vector=torch.sum(after_softmax*outputs,dim=1) # ( batch_size, dense_dim )

    return context_vector # ( batch_size, dense_dim )

In [ ]:
a=torch.Tensor([[1,2,3],[4,5,6]])
print(a.size())
a=a.unsqueeze(1)
print(a.size())
print(a.expand(2,4,3)) # 크기가 1인 차원에 대해서만 가능

torch.Size([2, 3])
torch.Size([2, 1, 3])
tensor([[[1., 2., 3.],
         [1., 2., 3.],
         [1., 2., 3.],
         [1., 2., 3.]],

        [[4., 5., 6.],
         [4., 5., 6.],
         [4., 5., 6.],
         [4., 5., 6.]]])


In [ ]:
## Sum
a=torch.Tensor([[[1,2,3],[4,5,6]]])
b=torch.Tensor([[[1,2,3]]])
print(a.size())
print(b.size())
a+b

torch.Size([1, 2, 3])
torch.Size([1, 1, 3])


tensor([[[2., 4., 6.],
         [5., 7., 9.]]])

In [ ]:
## Multi token Sum
a=torch.Tensor([[[1,2,3],[4,5,6]]])
b=torch.Tensor([[[1,2,3]],[[4,5,6]]])
print(a.size())
print(b.size())
b+a

torch.Size([1, 2, 3])
torch.Size([2, 1, 3])


tensor([[[ 2.,  4.,  6.],
         [ 5.,  7.,  9.]],

        [[ 5.,  7.,  9.],
         [ 8., 10., 12.]]])

In [ ]:
## Softmax
softmax=nn.Softmax(dim=1)
a=torch.Tensor([[[3],[5],[4],[1]]])
out=softmax(a)
out

tensor([[[0.0889],
         [0.6572],
         [0.2418],
         [0.0120]]])

In [ ]:
## Weighted Sum
a=torch.Tensor([[[1,2,3],[4,5,6]]])
b=torch.Tensor([[[1],[2]]])
print(a.size())
print(b.size())
a*b

torch.Size([1, 2, 3])
torch.Size([1, 2, 1])


tensor([[[ 1.,  2.,  3.],
         [ 8., 10., 12.]]])

In [ ]:
class Decoder(nn.Module):
  def __init__(self,vocab_size,emb_dim,hidden_dim,dense_dim):
    super(Decoder,self).__init__()
    self.embedding=nn.Embedding(vocab_size,emb_dim)
    self.gru=nn.GRU(emb_dim+dense_dim,hidden_dim,batch_first=True)
    self.dense=nn.Linear(hidden_dim,vocab_size)
  
  def forward(self,x,context):
    # x : ( batch_size, k )    / k : 현재 Deocder에서 처리하고 있는 토큰의 index+1 # index=[0,max_length]
    # context : ( bath_size, k, dense_dim==emb_dim)
    after_emb=self.embedding(x) # ( batch_size, k , emb_dim)
    after_cat=torch.cat([context,after_emb],dim=-1)  # ( batch_size, k , emb_dim+dense_dim)
    output,state=self.gru(after_cat) # ( batch_size, k , hidden_dim), ( 1, batch_size, hidden_dim )
    state=torch.squeeze(state,dim=0) # ( batch_size, hidden_dim )
    predict_token=self.dense(output) # ( batch_size, k, hidden_dim) -> ( batch_size, k, vocab_size )
    return  predict_token, state # (batch_size, k, vocab_size) , ( batch_size, hidden_dim )

In [ ]:
a=torch.Tensor([[[1,2,3],[4,5,6]],[[3,2,1],[6,5,4]]])
b=torch.Tensor([[[1,2,3],[4,5,6]],[[4,5,6],[7,8,9]]])
print(a.size())
print(b.size())
torch.cat([b,a],dim=-1)

torch.Size([2, 2, 3])
torch.Size([2, 2, 3])


tensor([[[1., 2., 3., 1., 2., 3.],
         [4., 5., 6., 4., 5., 6.]],

        [[4., 5., 6., 3., 2., 1.],
         [7., 8., 9., 6., 5., 4.]]])

In [ ]:
a=torch.Tensor([[[1,2,3]],[[6,5,4]]])
b=torch.Tensor([[[1,2,3]],[[4,5,6]]])
print(a.size(),a,sep='\n')
print(b.size(),b,sep='\n')
torch.cat([a,b],dim=1)

torch.Size([2, 1, 3])
tensor([[[1., 2., 3.]],

        [[6., 5., 4.]]])
torch.Size([2, 1, 3])
tensor([[[1., 2., 3.]],

        [[4., 5., 6.]]])


tensor([[[1., 2., 3.],
         [1., 2., 3.]],

        [[6., 5., 4.],
         [4., 5., 6.]]])

In [ ]:
class Seq2seq(nn.Module):
  def __init__(self,vocab_size,emb_dim,hidden_dim,dense_dim):
    super(Seq2seq,self).__init__()
    self.encoder=Encoder(vocab_size,emb_dim,hidden_dim,dense_dim)
    self.attention=Attention(dense_dim)
    self.decoder=Decoder(vocab_size,emb_dim,hidden_dim,dense_dim)

  def forward(self,enc_input,dec_input): # enc_input : ( batch_size, max_len ) , dec_input : ( batch_size, max_ len )
    # outputs: ( batch_size, Max_len, dense_dim ) last_hidden_state : ( batch_size, hidden_dim )
    outputs,last_hidden_state=self.encoder(enc_input)
    context=self.attention(outputs,last_hidden_state) # ( batch_size, dense_dim )
    context=torch.unsqueeze(context,dim=1) # ( batch_size, 1, dense_dim ) <= 여기에다가 25개의 context_vector들을 모을 것임

    for i in range(1,MAX_LEN+1):
      predict,state=self.decoder(dec_input[:,:i],context)  # (batch_size, i, vocab_size) , ( batch_size, hidden_dim )
      new_context=self.attention(outputs,state) # ( batch_size, dense_dim )
      new_context=torch.unsqueeze(new_context,dim=1) # ( batch_size, 1, dense_dim )
      context=torch.cat([context,new_context],dim=1) # ( batch_size, i+1, dense_dim ) <= 다음 loop에서 context로 사용
    
    predict=predict.transpose(1,2)
    # 이렇게 돌고 맨 마지막으로 나온 25개의 연결된 값을 반환
    return predict # ( batch_size, vocab_size, max_len==25 )
    # -> 이에 대한 label : ( batch_size, max_len ) : crossenctorpyloss 사용하여 각각의 label에 해당하는 index(해당 단어의 index)의 요소의 값 가장 크도록 학습

In [ ]:
a=torch.Tensor([[1,2,3],[4,5,6]])
a=torch.unsqueeze(a,dim=1)
print(a.size(),a,sep='\n')

torch.Size([2, 1, 3])
tensor([[[1., 2., 3.]],

        [[4., 5., 6.]]])


In [ ]:
a=torch.Tensor([[1,2,3],[4,5,6]])
a.size()
a[:,:2]

tensor([[1., 2.],
        [4., 5.]])

### 모델 컴파일

In [ ]:
BATCH_SIZE=2
MAX_LEN=25
EPOCH=50
UNIT=1024
EMB_DIM=256
lr=1e-3
device='cuda' if torch.cuda.is_available() else 'cpu'
vocab_size=len(V.vocab)
vocab_size

2907

In [ ]:
model=Seq2seq(vocab_size,EMB_DIM,UNIT,UNIT).to(device)
optimizer=torch.optim.Adam(model.parameters(),lr)
criterion=nn.CrossEntropyLoss()

In [ ]:
predict=Seq2seq(vocab_size,EMB_DIM,UNIT,UNIT).to(device)
predict=predict(batch.question.to(device),batch.answer.to(device))
criterion(predict,batch.label.to(device))

tensor(8.0266, device='cuda:0', grad_fn=<NllLoss2DBackward0>)

In [ ]:
a=torch.Tensor([[[1,2,3],[4,5,6]],[[7,8,9],[3,2,1]]])
print(a)
print(a.size())
a=a.transpose(1,2)
print(a)
print(a.size())

tensor([[[1., 2., 3.],
         [4., 5., 6.]],

        [[7., 8., 9.],
         [3., 2., 1.]]])
torch.Size([2, 2, 3])
tensor([[[1., 4.],
         [2., 5.],
         [3., 6.]],

        [[7., 3.],
         [8., 2.],
         [9., 1.]]])
torch.Size([2, 3, 2])


## 모델 학습

In [ ]:
from tqdm import tqdm

In [128]:
def train(model,optimizer,criterion,train_iter):
  model.train()

  for batch in tqdm(train_iter):
    if batch.question.size()[0]!=2:
      break
    enc_input=batch.question.to(device)
    dec_input=batch.answer.to(device)
    label=batch.label.to(device)

    logit=model(enc_input,dec_input)
    loss=criterion(logit,label)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

In [129]:
def evaluation(model,optimizer,criterion,val_iter):
  model.eval()

  total_loss,acc=0,0
  for batch in tqdm(val_iter):
    if batch.question.size()[0]!=2:
      break
    enc_input=batch.question.to(device)
    dec_input=batch.answer.to(device)
    label=batch.label.to(device)

    logit=model(enc_input,dec_input)
    loss=criterion(logit,label)  # logit : (batch_size, vocab_size, max_len) , label : (batch_size, max_len)
    total_loss+=loss
    logit=logit.transpose(1,2) # logit : (batch_size, max_len, vocab_size)
    acc+=(logit.max(2)[1]==label).sum()
  size=len(val_iter.dataset)
  avg_loss=total_loss/size
  avg_acc=acc/size

  return avg_loss,avg_acc

In [ ]:
a=torch.Tensor([[[1,2,3,4],[5,6,7,8]],[[4,3,2,1],[9,8,7,6]]])
a.max(2)[1]

tensor([[3, 3],
        [0, 0]])

In [ ]:
import os

In [131]:
best_val_loss=None

for e in range(1,EPOCH+1):
  train(model,optimizer,criterion,train_iter)
  val_loss,val_acc=evaluation(model,optimizer,criterion,val_iter)

  print("Epoch : {} | val_loss : {:5.2f} , val_accuracy : {:5.2f}".format(e,val_loss,val_acc))

  if not best_val_loss or best_val_loss>val_loss:
    os.makedirs('best_weight',exist_ok=True)
    torch.save(model.state_dict(),'./best_weight/best_performance_weight.pt')
    best_val_loss=val_loss

  2%|▏         | 111/5321 [00:10<08:14, 10.53it/s]


KeyboardInterrupt: ignored